# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
# create file path
infile = os.path.join("Output", "city_weather_data.csv")

#read csv into a dataframe
city_data = pd.read_csv(infile)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
gmaps.configure(api_key=g_key)

# Create a variable for lat, long
city_locations = city_data[["Lat", "Lng"]]

# Create a variable for humidity'
city_humidity = city_data["Humidity"]

# Create the  figure
figure_layout = {
    'width': '800px',
    'height': '580px',
    'border': '3px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(zoom_level = 2, center = (24.288292, 11.701573), layout=figure_layout)

# Create the heatmap layer for city locations and humidity
heat_layer = gmaps.heatmap_layer(city_locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=city_humidity.max(),
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(border='3px solid black', height='580px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
city_data_filtered = city_data.loc[(city_data['Max Temp'].between(70, 80)) & (city_data['Wind Speed'] < 5) & (city_data['Cloudiness'] < 5)].copy()  
city_data_clean = city_data_filtered.dropna()
city_data_clean

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
119,Nisibis,37.08,41.22,74.80,30,0,1.57,TR,1600385709
193,Buritizeiro,-17.35,-44.96,73.24,34,0,1.41,BR,1600385751
255,Rio de Janeiro,-22.90,-43.21,75.20,83,1,4.70,BR,1600385558
382,Dzaoudzi,-12.79,45.27,71.60,88,2,3.36,YT,1600385847
401,Sirte,31.21,16.59,74.98,75,0,2.28,LY,1600385862
415,Ajdabiya,30.76,20.23,70.75,86,0,4.23,LY,1600385865
463,Manavgat,36.79,31.44,75.20,83,0,2.91,TR,1600385886
491,Piaçabuçu,-10.41,-36.43,72.54,81,3,4.68,BR,1600385893
509,Dom Pedro,-4.48,-44.45,79.38,57,2,3.44,BR,1600385908
538,Inhambane,-23.86,35.38,70.79,89,2,4.36,MZ,1600385926


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = city_data_clean
hotel_df = hotel_df.reset_index(drop=True)

In [33]:
hotel_df['Hotel Name'] = ""
hotel_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Create a dictionary variable to store the parameters
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key,
}

for index, row in hotel_df.iterrows():

    # get restaurant type from DataFrame
    lat_longs = f"{row['Lat']},{row['Lng']}"

    # add a new key to our params dictionary that will store our keyword to search for
    params['location'] = lat_longs

    # assemble url and make API request
#    print(f"Retrieving Results for Index {index}: {row['City']} {lat_longs}.")
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"The closest hotel in {row['City']} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print(f"No hotel found in {row['City']}")

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Nisibis,37.08,41.22,74.80,30,0,1.57,TR,1600385709,Aydinlar Hotel
1,Buritizeiro,-17.35,-44.96,73.24,34,0,1.41,BR,1600385751,Santos Palace Hotel
2,Rio de Janeiro,-22.90,-43.21,75.20,83,1,4.70,BR,1600385558,Windsor Guanabara Hotel
3,Dzaoudzi,-12.79,45.27,71.60,88,2,3.36,YT,1600385847,Le Rocher
4,Sirte,31.21,16.59,74.98,75,0,2.28,LY,1600385862,City Hotel
5,Ajdabiya,30.76,20.23,70.75,86,0,4.23,LY,1600385865,فندق التوجان
6,Manavgat,36.79,31.44,75.20,83,0,2.91,TR,1600385886,Lake & River Side Hotel & Spa
7,Piaçabuçu,-10.41,-36.43,72.54,81,3,4.68,BR,1600385893,Piaçabuçu News
8,Dom Pedro,-4.48,-44.45,79.38,57,2,3.44,BR,1600385908,Pousada Santa Amália
9,Inhambane,-23.86,35.38,70.79,89,2,4.36,MZ,1600385926,Oceano


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [32]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the figure
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='3px solid black', height='580px', margin='0 auto 0 auto', padding='1px', wi…